In [15]:
%pip -q install google-genai

In [16]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [17]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [18]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk


In [19]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [20]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [21]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##### ⚠️⚠️⚠️ ATENÇÃO: APÓS EXECUTAR ESSA CÉLULA, INSIRA OS ARQUIVOS EM PDF DAS FATURAS NA PASTA SOLICITADA ⚠️⚠️⚠️ #####
##########################################################################################################################
# --- Instalação de Dependências ---
%pip install -q google-genai PyPDF2

# --- Importações ---
import os
import pandas as pd
from google.colab import drive
from google.colab import userdata
import google.generativeai as genai
import PyPDF2
from IPython.display import display, Markdown
import textwrap
import warnings
import locale
import re

warnings.filterwarnings("ignore")


# --- Configuração da API Key do Google Gemini ---
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
except Exception as e:
    print(f"❗ Erro ao configurar a API Key: {e}")
    print("Certifique-se de que a chave 'GOOGLE_API_KEY' está configurada nos segredos do Colab.")
    exit(1)

# --- Montagem do Google Drive ---
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive')
        print("Google Drive montado com sucesso.")
    except Exception as e:
        print(f"❗ Erro ao montar o Google Drive: {e}")
        print("❗ Verifique sua conexão com o Google Drive e tente novamente.")
        exit(1)
else:
    print("Google Drive já está montado.")

# --- Configuração do Caminho Base ---
PASTA_BASE_CRIAÇÃO = '/content/drive/MyDrive/CondominAI'
nome_nova_pasta = "Faturas de Condomínio"

# --- Criar as Pastas ---
if not nome_nova_pasta:
    print("Nome da pasta não pode ser vazio. Operação cancelada.")
    exit(1)
else:
    caminho_completo_nova_pasta = os.path.join(PASTA_BASE_CRIAÇÃO, nome_nova_pasta)
    try:
        os.makedirs(caminho_completo_nova_pasta, exist_ok=True)
        print(f"\n✅ Sucesso! A pasta '{nome_nova_pasta}' foi criada no seu Google Drive.")
        print(f"Caminho completo no Colab (faturas): {caminho_completo_nova_pasta}")
        print(f"⚠️⚠️ Atenção: Antes de prosseguir, insira os arquivos em PDF das faturas na pasta '{nome_nova_pasta}'.")
    except FileNotFoundError:
        print(f"\nErro: O caminho base '{PASTA_BASE_CRIAÇÃO}' não foi encontrado ou não está acessível.")
        print("Verifique se o Google Drive está montado corretamente e se o caminho base existe.")
        exit(1)
    except Exception as e:
        print(f"\n🚨 Ocorreu um erro ao criar as pastas: {e}")
        print("Verifique as permissões de escrita no Google Drive ou se os nomes das pastas contêm caracteres inválidos.")
        exit(1)

# --- Configuração do Caminho da Pasta de Faturas ---
PASTA_FATURAS_DRIVE = caminho_completo_nova_pasta

# --- Função Auxiliar para Exibir Texto em Markdown ---
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# --- Função para Listar Arquivos PDF ---
def listar_faturas_pdf(pasta_drive):
    lista_arquivos = []
    try:
        if os.path.exists(pasta_drive):
            conteudo = os.listdir(pasta_drive)
            print(f"Conteúdo da pasta {pasta_drive}: {conteudo}")
            for nome_arquivo in conteudo:
                if nome_arquivo.lower().endswith('.pdf'):
                    caminho_completo = os.path.join(pasta_drive, nome_arquivo)
                    lista_arquivos.append(caminho_completo)
            if not lista_arquivos:
                print(f"❗ Nenhum arquivo PDF encontrado em {pasta_drive}")
        else:
            print(f"❗ Pasta {pasta_drive} não existe.")
    except Exception as e:
        print(f"❗ Erro ao listar arquivos em {pasta_drive}: {e}")
    return lista_arquivos

# --- Função Auxiliar para Chamar o Modelo Gemini ---
def call_agent(model_name: str, message_text: str) -> str:
    print(f"Chamando Gemini: {model_name}")
    if not isinstance(model_name, str):
        raise ValueError(f"model_name deve ser uma string, recebido: {type(model_name)}")
    try:
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(message_text)
        return response.text
    except Exception as e:
        print(f"Erro ao chamar o Gemini: {e}")
        return ""

# --- Função para Ler Texto de PDF ---
def ler_texto_pdf(caminho_arquivo_pdf, page_num=None):
    texto = ""
    try:
        with open(caminho_arquivo_pdf, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            if page_num is not None:
                if 0 <= page_num < len(reader.pages):
                    texto_pagina = reader.pages[page_num].extract_text()
                    if texto_pagina:
                        texto = texto_pagina
                else:
                    return None
            else:
                for page in reader.pages:
                    texto_pagina = page.extract_text()
                    if texto_pagina:
                        texto += texto_pagina
        return texto
    except Exception as e:
        print(f"Erro ao ler PDF {caminho_arquivo_pdf}: {e}")
        return None

# --- Função para Formatação Monetária ---
def formatar_valor_monetario(valor):
    if valor is None or valor == "N/A":
        return "N/A"
    try:
        valor_float = float(valor.replace('.', '').replace('.', ',') if isinstance(valor, str) else valor)
        try:
            return locale.currency(valor_float, grouping=True, symbol=None)
        except locale.Error:
            valor_str = f"{valor_float:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
            return valor_str
    except (ValueError, TypeError):
        return valor

# --- Função para Normalizar Chaves ---
def normalizar_chave(texto):
    return re.sub(r'[^a-z0-9_]', '', texto.lower().replace(' ', '_'))

# --- Função para Verificar se é um Valor Numérico ---
def is_numeric_value(value):
    if not isinstance(value, str) or value == "N/A":
        return False
    # Verifica se o valor corresponde a um número (ex.: "63.630,00" ou "63630,00")
    return bool(re.match(r'^-?\d{1,3}(?:\.\d{3})*(?:,\d{2})?$', value))

# --- Função para Parsear Texto do Gemini ---
def parsear_texto_em_dicionario(texto_gemini):
    dados = {}
    chave_original_map = {}  # Mapeia chaves normalizadas para textos originais
    categoria_atual = None
    linhas = texto_gemini.strip().split('\n')

    for linha in linhas:
        linha = linha.strip()
        if not linha or linha.startswith('---'):
            categoria_atual = None
            continue

        if ':' in linha:
            chave, valor = linha.split(':', 1)
            chave_original = chave.strip()  # Preservar texto original
            valor = valor.strip()

            # Normalizar chave
            chave_formatada = normalizar_chave(chave_original)
            chave_original_map[chave_formatada] = chave_original  # Mapear normalizada -> original

            # Verificar se é uma categoria principal ou subcategoria
            if valor.replace('.', '').replace(',', '').isdigit() or valor.lower() == 'n/a' or '(' in valor:
                # É provavelmente um item com valor ou percentual
                if categoria_atual:
                    # Adicionar como subcategoria da categoria atual
                    if f"{categoria_atual}_detalhes" not in dados:
                        dados[f"{categoria_atual}_detalhes"] = {}
                        chave_original_map[f"{categoria_atual}_detalhes"] = chave_original_map[categoria_atual] + " Detalhes"
                    dados[f"{categoria_atual}_detalhes"][chave_original] = valor
                else:
                    # Adicionar como campo principal
                    dados[chave_formatada] = valor
            else:
                # É provavelmente uma categoria principal
                categoria_atual = chave_formatada
                dados[chave_formatada] = valor
                if f"{chave_formatada}_detalhes" not in dados:
                    dados[f"{chave_formatada}_detalhes"] = {}
                    chave_original_map[f"{chave_formatada}_detalhes"] = chave_original + " Detalhes"

    return dados, chave_original_map

In [23]:
##########################################
# ---  Agente 1: Coletor de Faturas  --- #
##########################################

def agente_coletor_faturas():
    lista_caminhos = listar_faturas_pdf(PASTA_FATURAS_DRIVE)
    if not lista_caminhos:
        mensagem = "🚨 Nenhuma fatura PDF encontrada na pasta do Google Drive."
        print("\n--- 📂 Resultado do Agente 1 (Coletor de Faturas) ---\n")
        display(to_markdown(mensagem))
        return [], None

    nomes_arquivos = [os.path.basename(caminho) for caminho in lista_caminhos]
    prompt = f"""
    Você é um agente que lista arquivos de faturas de condomínio disponíveis.
    Sua tarefa é receber uma lista de nomes de arquivos PDF e retornar uma mensagem formatada
    confirmando os arquivos encontrados, mantendo os nomes EXATAMENTE como fornecidos.

    Lista de arquivos PDF:
    {', '.join(nomes_arquivos)}

    Retorne uma mensagem no formato:
    Faturas encontradas: {', '.join(nomes_arquivos)}.
    """
    resposta_agente = call_agent("gemini-2.0-flash", prompt)
    print("\n--- 📂 Resultado do Agente 1 (Coletor de Faturas) ---\n")
    display(to_markdown(resposta_agente))

    # Perguntar ao usuário o modo de operação
    print("\nO que você deseja fazer?")
    print("1. Analisar uma única fatura")
    print("2. Comparar duas ou mais faturas")
    while True:
        try:
            modo = int(input("⚠️ Escolha o modo (1 para análise única, 2 para comparação): "))
            if modo in [1, 2]:
                break
            print("❗ Modo inválido. Escolha 1 ou 2.")
        except ValueError:
            print("❗ Entrada inválida. Digite um número (1 ou 2).")
    return lista_caminhos, modo

In [24]:
#############################################
# ---  Agente 2: Analisador de Faturas  --- #
#############################################

def agente_analisador_fatura_mes(caminho_arquivo_fatura, mes_referencia):
    conteudo_texto_fatura = ler_texto_pdf(caminho_arquivo_fatura)
    if conteudo_texto_fatura is None:
        return {"erro": f"Não foi possível ler o conteúdo da fatura em {caminho_arquivo_fatura}."}

    prompt = f"""
    Você é um especialista em análise de faturas de condomínio. Sua tarefa é extrair todas as informações relevantes
    do conteúdo de texto bruto de uma fatura, sem assumir uma estrutura predefinida. A fatura é referente ao mês de **{mes_referencia}**.

    **Instruções**:
    - Extraia os valores numéricos EXATAMENTE como aparecem na fatura, incluindo pontos e vírgulas (ex.: 20.766.215,00).
    - Se um valor não for encontrado, use "N/A".
    - Identifique o campo "Valor do Doc", que representa o valor total a ser pago pelo boleto do condomínio.
    - Localize seções como "Rateio" ou "Despesas" e extraia todas as categorias principais, subcategorias, valores e percentuais (%), se disponíveis.
    - Não assuma categorias predefinidas; extraia apenas o que está presente no texto.
    - Para cada categoria principal, liste suas subcategorias e valores associados.
    - Inclua campos gerais como "Mês Referência", "Data Vencimento", "Valor Total Geral" e "Valor Total Rateio", se encontrados.

    **Formato de saída**:
    Mês Referência: {mes_referencia}
    Data Vencimento: [dd/mm/yyyy ou N/A]
    Valor Total Geral: [Valor, ex: 20.766.215,00 ou N/A]
    Valor Total Rateio: [Valor, ex: 18.362.507,00 ou N/A]
    Valor da Fatura: [Valor, ex: 63.630,00 ou N/A]
    ---
    [Categoria Principal 1]: [Valor Total ou N/A]
    [Subcategoria 1]: [Valor ou N/A] ([Percentual, ex: 10,5% ou N/A])
    [Subcategoria 2]: [Valor ou N/A] ([Percentual ou N/A])
    ---
    [Categoria Principal 2]: [Valor Total ou N/A]
    [Subcategoria 1]: [Valor ou N/A] ([Percentual ou N/A])
    ...

    **Texto da fatura**:
    {conteudo_texto_fatura}
    """
    resposta_agente = call_agent("gemini-2.0-flash", prompt)
    if not resposta_agente:
        return {"erro": f"Falha ao extrair dados da fatura em {caminho_arquivo_fatura}."}

    dados_extraidos, chave_original_map = parsear_texto_em_dicionario(resposta_agente)
    dados_extraidos['arquivo_origem'] = os.path.basename(caminho_arquivo_fatura)
    dados_extraidos['mes_referencia_validado'] = mes_referencia
    chave_original_map['arquivo_origem'] = "Arquivo Origem"
    chave_original_map['mes_referencia_validado'] = "Mês Referência Validado"

    def exibir_resultado(dados, chave_map):
        def formatar_dicionario(d, indent=0, prefix="", chave_map=None):
            resultado = []
            for chave, valor in d.items():
                display_key = chave_map.get(chave, chave.replace('_', ' ').title())
                if isinstance(valor, dict):
                    resultado.append("  " * indent + f"{display_key}:")
                    resultado.extend(formatar_dicionario(valor, indent + 1, prefix=f"{chave}_", chave_map=chave_map))
                else:
                    valor_formatado = formatar_valor_monetario(valor)
                    resultado.append("  " * indent + f"{display_key}: {valor_formatado}")
            return resultado

        print("\n--- Resultado do Agente 2 (Analisador de Faturas) ---\n")
        print("\n".join(formatar_dicionario(dados, indent=0, prefix="", chave_map=chave_map)))

    exibir_resultado(dados_extraidos, chave_original_map)
    return dados_extraidos, chave_original_map

In [25]:
############################################
# ---  Agente 3: Resumidor de Faturas  --- #
############################################

def agente_resumidor_faturas(dados_fatura_1, chave_original_map_1, lista_dados_faturas_adicionais=None):
    if not lista_dados_faturas_adicionais:
        # Resumo para uma única fatura
        print("\n--- Resultado do Agente 3 (Resumidor de Faturas) ---\n")
        print(f"📋 Resumo da Fatura: {dados_fatura_1['mes_referencia_validado']}\n")

        # Exibir valores principais
        print("📊 Principais Valores:")
        valores_principais = []
        for chave, valor in dados_fatura_1.items():
            if chave not in ['arquivo_origem', 'mes_referencia_validado'] and not chave.endswith('_detalhes'):
                valores_principais.append((chave, valor))
                display_key = chave_original_map_1.get(chave, chave.replace('_', ' ').title())
                print(f"  - {display_key}: {formatar_valor_monetario(valor)}")

        # Calcular distribuição de custos (percentuais em relação ao Valor do Doc)
        valor_doc = dados_fatura_1.get('valor_do_doc', 'N/A')
        distribuicao = []
        if valor_doc != "N/A" and is_numeric_value(valor_doc):
            try:
                valor_doc_float = float(valor_doc.replace('.', '').replace(',', '.'))
                print("\n📊 Distribuição de Custos:")
                for chave, valor in dados_fatura_1.items():
                    if chave.endswith('_detalhes'):
                        categoria = chave.replace('_detalhes', '')
                        display_categoria = chave_original_map_1.get(chave, categoria.replace('_', ' ').title())
                        detalhes = dados_fatura_1[chave]
                        for subchave, subvalor in detalhes.items():
                            if subvalor != "N/A" and is_numeric_value(subvalor):
                                try:
                                    subvalor_float = float(subvalor.replace('.', '').replace(',', '.'))
                                    percentual = (subvalor_float / valor_doc_float) * 100
                                    distribuicao.append((f"{display_categoria} - {subchave}", subvalor, percentual))
                                    print(f"  - {display_categoria} - {subchave}: {formatar_valor_monetario(subvalor)} ({percentual:.2f}% do Valor do Doc)")
                                except (ValueError, TypeError):
                                    continue
            except (ValueError, TypeError):
                print("  - Não foi possível calcular percentuais devido a valores inválidos.")

        # Observação geral (maior despesa)
        valores = {}
        for k, v in dados_fatura_1.items():
            if (k not in ['arquivo_origem', 'mes_referencia_validado'] and
                not k.endswith('_detalhes') and
                v != "N/A" and
                is_numeric_value(v)):
                try:
                    valores[k] = float(v.replace('.', '').replace('.', ','))
                except (ValueError, TypeError):
                    continue
        if valores:
            maior_categoria = max(valores, key=valores.get)
            display_key = chave_original_map_1.get(maior_categoria, maior_categoria.replace('_', ' ').title())
            print(f"\n📝 Observação: {display_key} é a maior despesa, com {formatar_valor_monetario(dados_fatura_1[maior_categoria])}.")
        else:
            print("\n📝 Observação: Não foi possível identificar a maior despesa devido a valores inválidos ou ausentes.")

        return None  # Nenhum dicionário de comparação

    # Comparação para múltiplas faturas
    comparacoes = []
    for dados_fatura_2, chave_original_map_2 in lista_dados_faturas_adicionais:
        comparacao = {
            "mes_1": dados_fatura_1["mes_referencia_validado"],
            "mes_2": dados_fatura_2["mes_referencia_validado"],
            "diferencas": {},
            "inclusoes": [],
            "exclusoes": []
        }

        # Identificar todas as chaves (categorias e subcategorias) de ambos os dicionários
        chaves_1 = set(dados_fatura_1.keys())
        chaves_2 = set(dados_fatura_2.keys())
        todas_chaves = chaves_1.union(chaves_2)

        # Comparar campos principais (excluindo detalhes e metadados)
        campos_principais = [chave for chave in todas_chaves if not chave.endswith('_detalhes') and chave not in ['arquivo_origem', 'mes_referencia_validado']]

        for campo in campos_principais:
            valor_1_str = dados_fatura_1.get(campo, "N/A")
            valor_2_str = dados_fatura_2.get(campo, "N/A")

            # Converter para float, tratando N/A
            try:
                valor_1 = float(valor_1_str.replace('.', '').replace('.', ',')) if valor_1_str != "N/A" and is_numeric_value(valor_1_str) else 0.0
            except (ValueError, TypeError):
                valor_1 = 0.0
            try:
                valor_2 = float(valor_2_str.replace('.', '').replace('.', ',')) if valor_2_str != "N/A" and is_numeric_value(valor_2_str) else 0.0
            except (ValueError, TypeError):
                valor_2 = 0.0

            # Calcular diferença e percentual
            diferenca = valor_2 - valor_1
            percentual = ((valor_2 - valor_1) / valor_1 * 100) if valor_1 != 0 else (100.0 if valor_2 != 0 else 0.0)

            # Determinar tendência
            tendencia = "➡️"
            if abs(percentual) > 5.0:
                tendencia = "⬆️" if diferenca > 0 else "⬇️"
            grande_diferenca = abs(percentual) > 10.0

            comparacao["diferencas"][campo] = {
                "valor_1": formatar_valor_monetario(valor_1),
                "valor_2": formatar_valor_monetario(valor_2),
                "diferenca": formatar_valor_monetario(diferenca),
                "percentual": f"{percentual:.2f}%",
                "tendencia": tendencia,
                "grande_diferenca": grande_diferenca
            }

            # Identificar inclusões e exclusões
            if valor_1_str == "N/A" and valor_2_str != "N/A":
                display_key = chave_original_map_2.get(campo, campo.replace('_', ' ').title())
                comparacao["inclusoes"].append(f"{display_key}: {formatar_valor_monetario(valor_2_str)}")
            elif valor_1_str != "N/A" and valor_2_str == "N/A":
                display_key = chave_original_map_1.get(campo, campo.replace('_', ' ').title())
                comparacao["exclusoes"].append(f"{display_key}: {formatar_valor_monetario(valor_1_str)}")

        # Comparar subcategorias
        for chave in todas_chaves:
            if chave.endswith('_detalhes'):
                categoria = chave.replace('_detalhes', '')
                display_categoria_1 = chave_original_map_1.get(chave, categoria.replace('_', ' ').title())
                display_categoria_2 = chave_original_map_2.get(chave, categoria.replace('_', ' ').title())
                detalhes_1 = dados_fatura_1.get(chave, {})
                detalhes_2 = dados_fatura_2.get(chave, {})
                todas_subchaves = set(detalhes_1.keys()).union(set(detalhes_2.keys()))

                for subchave in todas_subchaves:
                    valor_1_str = detalhes_1.get(subchave, "N/A")
                    valor_2_str = detalhes_2.get(subchave, "N/A")

                    # Converter para float, tratando N/A
                    try:
                        valor_1 = float(valor_1_str.replace('.', '').replace('.', ',')) if valor_1_str != "N/A" and is_numeric_value(valor_1_str) else 0.0
                    except (ValueError, TypeError):
                        valor_1 = 0.0
                    try:
                        valor_2 = float(valor_2_str.replace('.', '').replace('.', ',')) if valor_2_str != "N/A" and is_numeric_value(valor_2_str) else 0.0
                    except (ValueError, TypeError):
                        valor_2 = 0.0

                    # Calcular diferença e percentual
                    diferenca = valor_2 - valor_1
                    percentual = ((valor_2 - valor_1) / valor_1 * 100) if valor_1 != 0 else (100.0 if valor_2 != 0 else 0.0)

                    # Determinar tendência
                    tendencia = "➡️"
                    if abs(percentual) > 5.0:
                        tendencia = "⬆️" if diferenca > 0 else "⬇️"
                    grande_diferenca = abs(percentual) > 10.0

                    # Adicionar à comparação com chave normalizada
                    subchave_normalizada = normalizar_chave(subchave)
                    chave_completa = f"{chave}_{subchave_normalizada}"
                    comparacao["diferencas"][chave_completa] = {
                        "valor_1": formatar_valor_monetario(valor_1),
                        "valor_2": formatar_valor_monetario(valor_2),
                        "diferenca": formatar_valor_monetario(diferenca),
                        "percentual": f"{percentual:.2f}%",
                        "tendencia": tendencia,
                        "grande_diferenca": grande_diferenca
                    }

                    # Identificar inclusões e exclusões
                    if valor_1_str == "N/A" and valor_2_str != "N/A":
                        comparacao["inclusoes"].append(f"{display_categoria_2} - {subchave}: {formatar_valor_monetario(valor_2_str)}")
                    elif valor_1_str != "N/A" and valor_2_str == "N/A":
                        comparacao["exclusoes"].append(f"{display_categoria_1} - {subchave}: {formatar_valor_monetario(valor_1_str)}")

        comparacoes.append(comparacao)

    # Exibir resultado para cada comparação
    for comparacao in comparacoes:
        print("\n--- Resultado do Agente 3 (Resumidor de Faturas) ---\n")
        print(f"📋 Resumo e Veredito: {comparacao['mes_1']} vs. {comparacao['mes_2']}\n")

        print(f"➕ Principais Itens Incluídos em {comparacao['mes_2']}:")
        if comparacao["inclusoes"]:
            for item in comparacao["inclusoes"][:5]:
                print(f"  - {item}")
            if len(comparacao["inclusoes"]) > 5:
                print(f"  - E mais {len(comparacao['inclusoes']) - 5} itens.")
        else:
            print("  Nenhum item novo incluído.")

        print(f"\n❌ Principais Itens Excluídos em {comparacao['mes_2']}:")
        if comparacao["exclusoes"]:
            for item in comparacao["exclusoes"][:5]:
                print(f"  - {item}")
            if len(comparacao["exclusoes"]) > 5:
                print(f"  - E mais {len(comparacao['exclusoes']) - 5} itens.")
        else:
            print("  Nenhum item excluído.")

        # Veredito
        print("\n📝 Conclusão:")
        grandes_diferencas = [campo for campo, info in comparacao["diferencas"].items() if info["grande_diferenca"]]
        if grandes_diferencas:
            print("Houve mudanças significativas entre as faturas:")
            for campo in grandes_diferencas[:3]:
                info = comparacao["diferencas"][campo]
                display_key = chave_original_map_1.get(campo, campo.replace('_', ' ').title())
                print(f"- {display_key}: {info['tendencia']} {info['percentual']} ({info['diferenca']})")
            if len(grandes_diferencas) > 3:
                print(f"- E mais {len(grandes_diferencas) - 3} categorias com grandes variações.")
            if "valor_do_doc" in grandes_diferencas:
                print("  * A variação no Valor do Doc indica uma mudança significativa no valor do boleto, impactando diretamente os condôminos.")
        else:
            print("As faturas apresentam variações pequenas, indicando estabilidade nas despesas do condomínio.")

    return comparacoes

In [26]:
##############################################
# ---  Agente 4: Pergunte ao CondominAI  --- #
##############################################

def agente_responsivo_perguntas(caminhos_faturas, dados_fatura_1, chave_original_map_1, lista_dados_faturas_adicionais=None, comparacoes=None):
    print("\n--- Agente 4: Respostas a Perguntas sobre Faturas ---\n")
    print("Digite sua pergunta sobre as faturas ou suas análises (digite 'sair' para encerrar):")

    # Mapear meses para caminhos de PDFs
    mes_to_caminho = {dados_fatura_1["mes_referencia_validado"].lower(): caminhos_faturas[0]}
    if lista_dados_faturas_adicionais:
        for i, (dados_fatura, _) in enumerate(lista_dados_faturas_adicionais, 1):
            mes_to_caminho[dados_fatura["mes_referencia_validado"].lower()] = caminhos_faturas[i]

    while True:
        pergunta = input("\nPergunta: ").strip()
        if pergunta.lower() == 'sair':
            print("Obrigado por utilizar o CondominAI. Até a próxima consulta! 😉")
            break

        if not pergunta:
            print("Estou pronto para ajudar! Qual pergunta você tem sobre a fatura? 🤔")
            continue

        # Preparar contexto para a pergunta
        contexto = f"""
        Você é um assistente especializado em responder perguntas sobre faturas de condomínio e suas análises.
        Você tem acesso aos seguintes dados:

        **Faturas Disponíveis**:
        - {dados_fatura_1['mes_referencia_validado']} (arquivo: {dados_fatura_1['arquivo_origem']})
        """
        if lista_dados_faturas_adicionais:
            for dados_fatura, _ in lista_dados_faturas_adicionais:
                contexto += f"- {dados_fatura['mes_referencia_validado']} (arquivo: {dados_fatura['arquivo_origem']})\n"

        contexto += f"""
        **Dados Extraídos ({dados_fatura_1['mes_referencia_validado']})**:
        {str(dados_fatura_1)}
        """
        if lista_dados_faturas_adicionais:
            for dados_fatura, _ in lista_dados_faturas_adicionais:
                contexto += f"""
        **Dados Extraídos ({dados_fatura['mes_referencia_validado']})**:
        {str(dados_fatura)}
        """
        if comparacoes:
            for comparacao in comparacoes:
                contexto += f"""
        **Comparação entre {comparacao['mes_1']} e {comparacao['mes_2']}**:
        - Diferenças: {str(comparacao['diferencas'])}
        - Itens Incluídos em {comparacao['mes_2']}: {str(comparacao['inclusoes'])}
        - Itens Excluídos em {comparacao['mes_2']}: {str(comparacao['exclusoes'])}
        """

        contexto += f"""
        **Instruções**:
        - Responda à pergunta do usuário com base nos dados fornecidos.
        - Se a pergunta mencionar um mês (ex.: 'Março', 'Abril'), associe ao respectivo arquivo de fatura.
        - Se a pergunta for sobre o conteúdo bruto do PDF (ex.: 'O que está na página 2?'), indique que você pode acessar o texto do PDF, mas não execute a leitura diretamente aqui.
        - Para valores monetários, formate com duas casas decimais e use vírgula como separador decimal (ex.: 63.630,00).
        - Se a informação não estiver disponível, informe que os dados não foram encontrados.
        - Forneça respostas claras e concisas, no formato Markdown.

        **Pergunta do usuário**:
        {pergunta}
        """

        # Verificar se a pergunta menciona uma página específica do PDF
        page_match = re.search(r'página\s+(\d+)|page\s+(\d+)', pergunta, re.IGNORECASE)
        if page_match:
            page_num = int(page_match.group(1) or page_match.group(2)) - 1
            mes_mencionado = None
            for mes in mes_to_caminho:
                if mes.lower() in pergunta.lower():
                    mes_mencionado = mes
                    break
            if mes_mencionado:
                caminho_pdf = mes_to_caminho[mes_mencionado]
                texto_pagina = ler_texto_pdf(caminho_pdf, page_num)
                if texto_pagina:
                    prompt_pagina = f"""
                    O usuário perguntou sobre o conteúdo da página {page_num + 1} do PDF da fatura de {mes_mencionado}.
                    Aqui está o texto extraído da página:
                    {texto_pagina[:2000]}  # Limitar para evitar excesso de tokens

                    Resuma o conteúdo da página em até 100 palavras e responda à pergunta:
                    {pergunta}
                    """
                    resposta = call_agent("gemini-2.5-pro-preview-03-25", prompt_pagina)
                else:
                    resposta = f"Não foi possível extrair o texto da página {page_num + 1} do PDF de {mes_mencionado}."
            else:
                resposta = "Por favor, especifique o mês da fatura (ex.: 'Março' ou 'Abril') para acessar a página."
        else:
            resposta = call_agent("gemini-2.0-flash", contexto)

        print("\n--- 🤖 Resposta do Agente 4 ---\n")
        display(to_markdown(resposta))

In [ ]:
#############################
# ---  Fluxo Principal  --- #
#############################

def main():
    # Executar Agente 1
    print("\n📂🕵️‍♂️ Executando Agente 1: Coletor de Faturas...")
    caminhos_faturas, modo = agente_coletor_faturas()
    if not caminhos_faturas:
        print("🛑 Nenhuma fatura encontrada. Finalizando.")
        return

    # Listar faturas disponíveis
    print("\n🗳️ Faturas disponíveis:")
    for i, caminho in enumerate(caminhos_faturas, 1):
        print(f"{i}. {os.path.basename(caminho)}")

    if modo == 1:  # Análise de uma única fatura
        if len(caminhos_faturas) < 1:
            print("🛑 Pelo menos uma fatura é necessária para análise. Finalizando.")
            return

        # Selecionar uma fatura
        print("\n👆📑 Selecione a fatura para análise:")
        while True:
            try:
                escolha = int(input("Digite o número da fatura (ex.: 1): ")) - 1
                if 0 <= escolha < len(caminhos_faturas):
                    break
                print(f"❗ Escolha inválida. Digite um número entre 1 e {len(caminhos_faturas)}.")
            except ValueError:
                print("❗ Entrada inválida. Digite um número.")

        caminho_fatura = caminhos_faturas[escolha]
        nome_arquivo = os.path.basename(caminho_fatura).lower()
        mes_referencia = "Não Identificado"
        meses = ["janeiro", "fevereiro", "março", "abril", "maio", "junho",
                 "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"]
        for mes in meses:
            if mes in nome_arquivo:
                mes_referencia = f"{mes.capitalize()} 2025"

        # Executar Agente 2
        print(f"\n🔍 Executando Agente 2: Analisador de Faturas para **{os.path.basename(caminho_fatura)}**...")
        # Capture the return value first
        agente2_result = agente_analisador_fatura_mes(caminho_fatura, mes_referencia)
        # Check if it's the error dictionary
        if "erro" in agente2_result:
            print(agente2_result["erro"])
            return # Exit if there was an error
        else:
            # Unpack only if it's not the error dictionary
            dados_fatura, chave_original_map = agente2_result


        # Executar Agente 3
        print("\n✍️ Executando Agente 3: Resumidor de Faturas...")
        agente_resumidor_faturas(dados_fatura, chave_original_map)

        # Executar Agente 4
        print("\n🤖 Executando Agente 4: Pergunte ao CondominAI...")
        agente_responsivo_perguntas([caminho_fatura], dados_fatura, chave_original_map)

    else:  # Comparação de duas ou mais faturas
        if len(caminhos_faturas) < 2:
            print("❗ Pelo menos duas faturas são necessárias para comparação. Finalizando.")
            return

        # Selecionar a primeira fatura
        print("\n👆📑 Selecione a primeira fatura para comparação:")
        while True:
            try:
                escolha_1 = int(input("Digite o número da fatura (ex.: 1): ")) - 1
                if 0 <= escolha_1 < len(caminhos_faturas):
                    break
                print(f"❗ Escolha inválida. Digite um número entre 1 e {len(caminhos_faturas)}.")
            except ValueError:
                print("❗ Entrada inválida. Digite um número.")

        # Selecionar faturas adicionais
        print("\n👆📑 Selecione as faturas adicionais para comparação (separadas por vírgula, ex.: 2,3):")
        while True:
            try:
                escolhas_adicionais = input("Digite os números das faturas: ").strip()
                escolhas_lista = [int(x) - 1 for x in escolhas_adicionais.split(',')]
                # Validar as escolhas
                validas = True
                if not escolhas_lista:
                    print("Pelo menos uma fatura adicional deve ser selecionada.")
                    validas = False
                elif len(escolhas_lista) != len(set(escolhas_lista)):
                    print("Não são permitidas faturas duplicadas.")
                    validas = False
                elif escolha_1 in escolhas_lista:
                    print("A primeira fatura não pode estar na lista de faturas adicionais.")
                    validas = False
                for escolha in escolhas_lista:
                    if not (0 <= escolha < len(caminhos_faturas)):
                        print(f"Escolha inválida: {escolha + 1}. Digite números entre 1 e {len(caminhos_faturas)}.")
                        validas = False
                if validas:
                    break
            except ValueError:
                print("🚨 Ops, entrada inválida. Digite números separados por vírgulas (ex.: 2,3).")

        caminho_fatura_1 = caminhos_faturas[escolha_1]
        caminhos_adicionais = [caminhos_faturas[i] for i in escolhas_lista]

        # Inferir mês de referência para a primeira fatura
        nome_arquivo_1 = os.path.basename(caminho_fatura_1).lower()
        mes_referencia_1 = "Não Identificado"
        meses = ["janeiro", "fevereiro", "março", "abril", "maio", "junho",
                 "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"]
        for mes in meses:
            if mes in nome_arquivo_1:
                mes_referencia_1 = f"{mes.capitalize()} 2025"

        # Executar Agente 2 para a primeira fatura
        print(f"\n🔍 Executando Agente 2: Analisador de Faturas para **{os.path.basename(caminho_fatura_1)}**...")
        # Capture the return value first
        agente2_result_1 = agente_analisador_fatura_mes(caminho_fatura_1, mes_referencia_1)
        # Check if it's the error dictionary
        if "erro" in agente2_result_1:
            print(agente2_result_1["erro"])
            return # Exit if there was an error
        else:
             # Unpack only if it's not the error dictionary
            dados_fatura_1, chave_original_map_1 = agente2_result_1

        # Executar Agente 2 para cada fatura adicional
        lista_dados_faturas_adicionais = []
        for caminho_fatura_2 in caminhos_adicionais:
            nome_arquivo_2 = os.path.basename(caminho_fatura_2).lower()
            mes_referencia_2 = "Não Identificado"
            for mes in meses:
                if mes in nome_arquivo_2:
                    mes_referencia_2 = f"{mes.capitalize()} 2025"

            print(f"\n🔍 Executando Agente 2: Analisador de Faturas para **{os.path.basename(caminho_fatura_2)}**...")
            # Capture the return value first
            agente2_result_2 = agente_analisador_fatura_mes(caminho_fatura_2, mes_referencia_2)
            # Check if it's the error dictionary
            if "erro" in agente2_result_2:
                print(agente2_result_2["erro"])
                # Decide how to handle the error for additional files.
                # For now, we'll print the error and continue with the successful ones,
                # or you might choose to skip this file or exit.
                continue # Skip this file and continue to the next one
            else:
                # Unpack only if it's not the error dictionary
                dados_fatura_2, chave_original_map_2 = agente2_result_2
                lista_dados_faturas_adicionais.append((dados_fatura_2, chave_original_map_2))


        # Check if we have at least one additional fatura for comparison
        if not lista_dados_faturas_adicionais and len(caminhos_adicionais) > 0:
             print("🛑 Nenhuma das faturas adicionais pôde ser processada. Finalizando.")
             return


        # Executar Agente 3
        print("\n✍️ Executando Agente 3: Resumidor de Faturas...")
        comparacoes = agente_resumidor_faturas(dados_fatura_1, chave_original_map_1, lista_dados_faturas_adicionais)

        # Executar Agente 4
        print("\n🤖 Executando Agente 4: Pergunte ao CondominAI...")
        # Pass the original list of all selected file paths to Agent 4
        agente_responsivo_perguntas([caminho_fatura_1] + caminhos_adicionais, dados_fatura_1, chave_original_map_1, lista_dados_faturas_adicionais, comparacoes)

if __name__ == "__main__":
    main()